In [1]:
#Entrada: los datos ya descargados de RASFF y las salidas del "Análisis_full_RASFF_Data".

# Preprocesamiento 

## Librerías

In [2]:
import numpy as np
import pandas as pd
import stellargraph as sg
import tensorflow as tf
import random 

from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import DeepGraphCNN
from stellargraph import StellarGraph

from sklearn import model_selection
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import confusion_matrix

from IPython.display import display, HTML

from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras import losses


## Carga del dataset

In [3]:
df = pd.read_csv('./../../Datasets/full_RASFF_DATA.csv', sep=';', header=0, index_col = 0)

#### Carga de los datasets auxiliares (tratamiento)

In [4]:
df_productos = pd.read_csv('./../../Datasets/Lista_Productos.csv', header=0, index_col = 0)
df_cat_productos = pd.read_csv('./../../Datasets/Lista_Categoria_Productos.csv', header=0, index_col = 0)
df_amenazas = pd.read_csv('./../../Datasets/Lista_Amenazas.csv', header=0, index_col = 0)
df_cat_amenazas = pd.read_csv('./../../Datasets/Lista_Categoria_Amenazas.csv', header=0, index_col = 0)
df_repes = pd.read_csv('./../../Datasets/Lista_repeticion_paises.csv', header=0, index_col = 0)

#### Conversión de NaN a formato string

In [5]:
df = df.replace(np.nan, "", regex=True)

## Corrección del dataset

In [6]:
#Seleccionamos la primera categoría de amenaza entre todas las posibles
for index, row in df.iterrows():
    row['HAZARDS_CAT'] = row['HAZARDS_CAT'].split(",")[0]

## Elección de fechas

In [7]:
#Eliminamos las fechas que no nos interesan.
fecha_maxima = "2021" #Primer año que no queremos coger.
df = df.loc[df['DATE_CASE'] < fecha_maxima]

## Agrupación de clases

In [8]:
#Segun el dendograma visto anteriormente, vamos a agrupar las clases 
#"labelling absent/incomplete/incorrect" con "packaging defective / incorrect"
#bajo una nueva clase llamada "labelling absent/packaging defective/incorrect"

for index, row in df.iterrows():
    if(row['HAZARDS_CAT'] == "labelling absent/incomplete/incorrect" or row['HAZARDS_CAT'] == "packaging defective / incorrect"):
        row['HAZARDS_CAT'] = "labelling absent/packaging defective/incorrect"

## Eliminamos los registros que no deseamos 
Las categorias obsoletas y los países con una tasa de participación inferior al 1%

In [9]:
paises_eliminar = df_repes.tail(72)
cat_productos_eliminar = df_cat_productos.tail(16)
cat_productos_eliminar = cat_productos_eliminar.append(df_cat_productos.iloc[-22])
cat_productos_eliminar = cat_productos_eliminar.sort_values('Repeticiones', ascending = False)
cat_amenazas_eliminar = df_cat_amenazas.tail(19) #Estaba en 19

In [10]:
#df

In [11]:
#df_cat_amenazas.tail(50)

In [12]:
#Eliminamos los registros que contienen valores no interesantes para nuestro estudio, mirando en las columnas de interés en cada registro.

#Guardamos las filas que no queremos coger en este datframe.
df_eliminar = df.drop(df.index, inplace=False)

#Buscamos esas filas "inútiles".
for index, row in df.iterrows():
    #Eliminamos los países invalidos.
    for j in (row["COUNT_ORIGEN"].split(",")):
        if (j in paises_eliminar['Pais'].values or j == "INFOSAN" or j == "Commission Services"):
            df_eliminar.loc[df_eliminar.shape[0]] = row
    for j in (row["COUNT_CONCERN"].split(",")):
        if (j in paises_eliminar['Pais'].values or j == "INFOSAN" or j == "Commission Services"):
            df_eliminar.loc[df_eliminar.shape[0]] = row
    for j in (row["COUNT_DESTIN"].split(",")):
        if (j in paises_eliminar['Pais'].values or j == "INFOSAN" or j == "Commission Services"):
            df_eliminar.loc[df_eliminar.shape[0]] = row
    #Eliminamos los productos inválidos.
    for j in (row["PROD_CAT"].split(",")):
        if (j in cat_productos_eliminar['Cat_Producto'].values):
            df_eliminar.loc[df_eliminar.shape[0]] = row
    #Eliminamos las categorías de amenazas inválidas.
    for j in (row["HAZARDS_CAT"].split(",")):
        if (j in cat_amenazas_eliminar['Cat_Amenaza'].values 
            or row["HAZARDS_CAT"] == ""
            or row["HAZARDS_CAT"] == " "):
            df_eliminar.loc[df_eliminar.shape[0]] = row
    #Eliminamos los registros que están vacios y no tienen información acerca de los paises.
    if((row["COUNT_ORIGEN"] == " " or row["COUNT_ORIGEN"] == "") 
       and (row["COUNT_CONCERN"] == " " or row["COUNT_CONCERN"] == "") 
       and (row["COUNT_DESTIN"] == " " or row["COUNT_DESTIN"] == "")):
        df_eliminar.loc[df_eliminar.shape[0]] = row
        
#Eliminamos por columna REF.
cond = df['REF'].isin(df_eliminar['REF'])
df.drop(df[cond].index, inplace = True)

## Undersampling y conjunto de test

In [13]:
#Debido al gran desequilibrio entre clases, equilibramos los datos con técnicas de undersampling.
num_max_clase = 1700      #Número de registros máximo por clase.
num_registros_test = 200  #Número de registros por clase para el conjunto de testeo.

#Creamos una lista con todos los dataframes según cada clase para hacfer el undersampling.
ans = [pd.DataFrame(y) for x, y in df.groupby('HAZARDS_CAT', as_index=False)]

#Creamos el conjunto de test por clases (sin seleccionar las columnas deseadas). Seleccionando 200 casos de todas las clases de forma random.
listaClasesTest = ans.copy()
for i in range(len(listaClasesTest)):
    listaClasesTest[i] = listaClasesTest[i].sample(n=num_registros_test)

#Escogemos la cantidad máxima de registros por cada clase.
for i in range(len(ans)):
    if(ans[i]['HAZARDS_CAT'].count() > num_max_clase):
        ans[i] = ans[i].sample(n=num_max_clase)
        
#Reunimos todos en un único datframe (df otra vez).
df = pd.concat(ans)
testRowData = pd.concat(listaClasesTest)

#Hacemos un shuffle para mezclar las clases entre sí.
df = df.sample(frac=1).reset_index(drop=True)

## Selección de atributos

In [14]:
#Creamos 2 datasets, uno con los parámetros a entrenar (df) y otro con la información relativa a cada registro (df_info)
df_info = df[['REF','PRODUCT','HAZARDS','DATE_CASE','CLASSIF','TYPE','RISK_DECISION', 'ACTION_TAKEN','DISTRIBUTION_STAT','NOT_COUNTRY']].reset_index(drop = True)
df = df[['PROD_CAT','HAZARDS_CAT','COUNT_ORIGEN','COUNT_CONCERN','COUNT_DESTIN']].reset_index(drop = True)

#De la misma forma con los datos del conjunto de test. QUEDA PENDIENTE HACER ESTO (TIENE QUE TENER MISMO FORMATO QUE LOS GRAFOS.)
testData = testRowData[['PROD_CAT','HAZARDS_CAT','COUNT_ORIGEN','COUNT_CONCERN','COUNT_DESTIN']].reset_index(drop = True)


## One hot encoding

In [15]:
#Sacamos el dataframe de las categorías de amenaza, que es lo que vamos a predecdf_nodes_featuressificar para
#poder compararlos y hacer un entrenamiento supervisado. Empleando OrdinalEncoder

ord_enc = OrdinalEncoder()

#Guardamos los valores temporalmente en y.
y = pd.DataFrame(columns = []) 
y["y_value"] = df['HAZARDS_CAT']
y["y_code"] = ord_enc.fit_transform(y[["y_value"]])

#Creamos un pequeño df con las conversiones, a modo de guía de qué codigo es qué valor.
y_guide = y.groupby(["y_value","y_code"]).sum()

#Pasamos al formatpo de graph_labels, que es el que se emplea para las GCNNs.
graph_labels = pd.Series(y["y_code"], dtype = "category", name = "Label")

df = df.drop('HAZARDS_CAT', axis=1)

In [17]:
#Hacemos el one hot encoding de las categorías de los productos del df.
x = pd.get_dummies(df['PROD_CAT'], prefix='PROD_CAT_')
#display(x)
#display(y.groupby("y_value").count())

In [19]:
#Convertimos el producto que corresponde al peso de los grafos
x = df['PROD_CAT'].astype('category').cat.codes
x = pd.DataFrame(data=x, columns = ['weight'], dtype = np.float32)
x['weight'] += 1 #Para que no haya un 0 como peso.
#print(x)


## Creación de los grafos

### Preparación

In [20]:
#Preparación previa. Elegimos los países que vamos a usar (correspondiendo a los que hemos eliminado más arriba).
paises = df_repes.head(159)['Pais'].sort_values().reset_index(drop = True)
paises = paises[1:]          #Quitamos los huecos vacíos " ".


In [21]:
#Matriz de características de los nodos 158x158.
df_nodes_features = pd.DataFrame(data = np.zeros((158, 158)), columns = paises)
s = pd.Series(data=np.ones(158))
np.fill_diagonal(df_nodes_features.values, s)

#Se podría probar a que fuese 158x1, con un 0 o 1 si tiene conexión o no. 
#El pais se fija en el index y al propia red debería abstarer esa información.


In [22]:
#Matriz de características de los nodos 158x1.

#Lista con todos los Node Features.
lista_df_node_features = []

#Bucle que recorra todos los registros, y relacione los nodos de df_nodes entre sí, marcando la relación entre sus índices.
#Además añadimos la x generada anteriormente.
for index, row in df.iterrows():
    #Creamos el df vacío, al que le añadimos las columnas source y target.
    arrayBase = np.zeros((158,1))
    df_nodes = pd.DataFrame(data = arrayBase, columns = ['Feature'])

    #Guardamos las longitudes de cada registro (nº de países). Si es vacío, establecemos un 0. 
    #LenCO = Country origen. LenCC = Country Concern y LenCD = Contry Destin.
    if(not row['COUNT_ORIGEN'] == "" and not row['COUNT_ORIGEN'] == " "):
        LenCO = len(row['COUNT_ORIGEN'].split(","))
        paisesCO = row['COUNT_ORIGEN'].split(",")
    else:
        LenCO = 0
    if(not row['COUNT_CONCERN'] == "" and not row['COUNT_CONCERN'] == " "):
        LenCC = len(row['COUNT_CONCERN'].split(","))
        paisesCC = row['COUNT_CONCERN'].split(",")
    else:
        LenCC = 0
    if(not row['COUNT_DESTIN'] == "" and not row['COUNT_DESTIN'] == " "):
        LenCD = len(row['COUNT_DESTIN'].split(","))
        paisesCD = row['COUNT_DESTIN'].split(",")
    else:
        LenCD = 0 

    #Rellenamos con 1s donde exista el país
    for i in range(LenCO):
        if(LenCO != 0):
            indexOrigen = df_nodes_features.index[df_nodes_features[paisesCO[i]] == True]
            df_nodes.iloc[indexOrigen] = 1
    for i in range(LenCC):
        if(LenCC != 0):
            indexOrigen = df_nodes_features.index[df_nodes_features[paisesCC[i]] == True]
            df_nodes.iloc[indexOrigen] = 1
    for i in range(LenCD):
        if(LenCD != 0):
            indexOrigen = df_nodes_features.index[df_nodes_features[paisesCD[i]] == True]
            df_nodes.iloc[indexOrigen] = 1
        
    #Insertamos el df en la lista de df_node_features.
    lista_df_node_features.append(df_nodes)

In [23]:
#Creamos los Edge Features -> 1 por grafo. (Representa la info de cada conexión).
#Columnas = Source, target y producto en one hot encoding. 
#Filas = nº de conexiones. 

#Lista con todos los Edge Features.
lista_df_edge_features = []

#Bucle que recorra todos los registros, y relacione los nodos de df_nodes entre sí, marcando la relación entre sus índices.
#Además añadimos la x generada anteriormente.
for index, row in df.iterrows():
    #Creamos el df vacío, al que le añadimos las columnas source y target.
    df_edges = pd.DataFrame(columns = x.columns)
    df_edges.insert (0, "target", np.nan)
    df_edges.insert (0, "source", np.nan)
    #Guardamos las longitudes de cada registro (nº de países). Si es vacío, establecemos un 0. 
    #LenCO = Country origen. LenCC = Country Concern y LenCD = Contry Destin.
    if(not row['COUNT_ORIGEN'] == "" and not row['COUNT_ORIGEN'] == " "):
        LenCO = len(row['COUNT_ORIGEN'].split(","))
        paisesCO = row['COUNT_ORIGEN'].split(",")
    else:
        LenCO = 0
    if(not row['COUNT_CONCERN'] == "" and not row['COUNT_CONCERN'] == " "):
        LenCC = len(row['COUNT_CONCERN'].split(","))
        paisesCC = row['COUNT_CONCERN'].split(",")
    else:
        LenCC = 0
    if(not row['COUNT_DESTIN'] == "" and not row['COUNT_DESTIN'] == " "):
        LenCD = len(row['COUNT_DESTIN'].split(","))
        paisesCD = row['COUNT_DESTIN'].split(",")
    else:
        LenCD = 0 
    #Creamos dos bucles anidados, para recorrer todos los source y linkarlos con todos los destinos. 
    #¡Esto funciona porque los índices de df_nodes siguen este mismo orden! 
    #Pero OJO! -> Existen 3 posibles casos: 1) LenCO = 0, 2) LenCC = 0 y 3) LenCD = 0. Teóricamente, habría que hacer 1->2->3.
    #Si los 3 existen y son mayores que 0 se ejecutan los dos bucles.
    
    #Empezamos a rellenar el df, si existe LenCO y LenCC, lo rellenamos. 
    if(LenCO != 0 and LenCC != 0):
        for i in range(LenCO):
            indexOrigen = df_nodes_features.index[df_nodes_features[paisesCO[i]] == True]
            for j in range(LenCC):
                indexDestino = df_nodes_features.index[df_nodes_features[paisesCC[j]] == True]
                serie = pd.Series([indexOrigen[0]] + [indexDestino[0]] + [np.float32(x.iloc[index][0])]) #.append(x.iloc[index])
                df_edges.loc[df_edges.shape[0]] = serie.values
                
    #Si LenCD es 0, no pasa nada, el grafo ya está creado. Si sí que existe, seguimos rellenando el grafo.
    if(LenCC != 0 and LenCD != 0):
        for i in range(LenCC):
            indexOrigen = df_nodes_features.index[df_nodes_features[paisesCC[i]] == True]
            for j in range(LenCD):
                indexDestino = df_nodes_features.index[df_nodes_features[paisesCD[j]] == True]
                serie = pd.Series([indexOrigen[0]] + [indexDestino[0]] + [np.float32(x.iloc[index][0])]) #.append(x.iloc[index])
                df_edges.loc[df_edges.shape[0]] = serie.values
                
    #Por aquí solo va a entrar si no ha entrado en los otros dos.
    if(LenCC == 0):
        for i in range(LenCO):
            indexOrigen = df_nodes_features.index[df_nodes_features[paisesCO[i]] == True]
            for j in range(LenCD):
                indexDestino = df_nodes_features.index[df_nodes_features[paisesCD[j]] == True]
                serie = pd.Series([indexOrigen[0]] + [indexDestino[0]] + [np.float32(x.iloc[index][0])]) #.append(x.iloc[index])
                df_edges.loc[df_edges.shape[0]] = serie.values
    
    #Insertamos el df en la lista de df_edge_features.
    lista_df_edge_features.append(df_edges)

#Convertimos la columna de weights a numérico, para que no de errores al crear los grafos de Stellargraph.
for i in range(len(lista_df_edge_features)):
    lista_df_edge_features[i].weight = pd.to_numeric(lista_df_edge_features[i].weight)

### Ejemplo visual

In [24]:
#Ejemplos para enseñar
miniIndex = 50
pd.set_option('display.max_columns', None)

display(df.iloc[miniIndex])
display(lista_df_node_features[miniIndex])
display(lista_df_edge_features[miniIndex])
graph_labels[miniIndex]
#y_guide

PROD_CAT         food contact materials
COUNT_ORIGEN                  Hong Kong
COUNT_CONCERN               China,Italy
COUNT_DESTIN                           
Name: 50, dtype: object

,Feature
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
153,0.0
154,0.0
155,0.0
156,0.0


,source,target,weight
0,63.0,27.0,13.0
1,63.0,74.0,13.0


7.0

### Creación de los grafos

In [25]:
#Creamos una lista con todos los StellarGraphs, pero los agrupamos de 10 en 10, hasta que se acabe una clase 

graph_labels_reducida = []  #Donde vamso a guardar los labels de los grafos resultantes.
lista_grafos = []           #Donde vamos a guardar los grafos resultantes.
posiciones = []             #Guardamos las posiciones de los ultimos 10 grafos con la msima amenaza

#Cuantos grafos queremos agrupar, si tenemos menos de n registros de esa amenaza dará error.
numGraphsCombined = 10

#No es el más óptimo, pero funciona.
for i in range(len(graph_labels.unique())):
    cont = 0
    df_nodes_prov = pd.DataFrame(data = np.zeros(158), columns = ["Feature"])
    df_edge_features_prov = pd.DataFrame(columns = ["source", "target", "weight"])
    for j in range(len(lista_df_edge_features)):
        if(graph_labels[j] == i):
            cont = cont + 1
            df_nodes_prov = df_nodes_prov.combine(lista_df_node_features[j],np.maximum)
            df_edge_features_prov = df_edge_features_prov.append(lista_df_edge_features[j],ignore_index=True)
            posiciones.append(j)
            if(cont%numGraphsCombined == 0):            
                #Insertamos el grafo de "numGraphsCombined" (combinado)
                lista_grafos.append(sg.StellarDiGraph(nodes = df_nodes_prov, edges = df_edge_features_prov, node_type_default="Paises", edge_type_default="Conexiones"))
                graph_labels_reducida.append(i)
                #Reseteamos los df auxiliares
                df_nodes_prov = pd.DataFrame(data = np.zeros(158), columns = ["Feature"])
                df_edge_features_prov = pd.DataFrame(columns = ["source", "target", "weight"])
    if(cont%10!=0):
        lista_grafos.append(sg.StellarDiGraph(nodes = df_nodes_prov, edges = df_edge_features_prov, node_type_default="Paises", edge_type_default="Conexiones"))
        graph_labels_reducida.append(i)


In [26]:
#Shuffle de lista_grafos y graph_labels_reducida

ObjetoProvisional = list(zip(lista_grafos, graph_labels_reducida))

random.shuffle(ObjetoProvisional)

lista_grafos, graph_labels_reducida = zip(*ObjetoProvisional)


In [27]:
display(len(lista_grafos))
display(len(graph_labels_reducida))

2223

2223

In [28]:
#Visualizamos info de un grafo
print(lista_grafos[8].info())

StellarDiGraph: Directed multigraph
 Nodes: 158, Edges: 21

 Node types:
  Paises: [158]
    Features: float32 vector, length 1
    Edge types: Paises-Conexiones->Paises

 Edge types:
    Paises-Conexiones->Paises: [21]
        Weights: range=[7, 23], mean=16.7143, std=5.34923
        Features: none


In [29]:
#Información acerca de los grafos obtenidos.
summary = pd.DataFrame(
    [(g.number_of_nodes(), g.number_of_edges()) for g in lista_grafos],
    columns=["nodes", "edges"],
)
summary.describe().round(1)

,nodes,edges
count,2223.0,2223.0
mean,158.0,19.0
std,0.0,9.2
min,158.0,9.0
25%,158.0,13.0
50%,158.0,16.0
75%,158.0,22.0
max,158.0,125.0


In [30]:
%store lista_grafos
%store graph_labels_reducida
%store y_guide

Stored 'lista_grafos' (tuple)
Stored 'graph_labels_reducida' (tuple)
Stored 'y_guide' (DataFrame)
